In [13]:
# Imports
import pandas as pd
import plotly.express as px 
import ipaddress
from urllib.request import urlopen
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [14]:
# read connection traffic data
#filename = '/Users/ZOHELHAS/Desktop/connection_traffic_data/luca_idle copy.csv'
filename = '/Users/Matteo-PC/Desktop/Assignment2_new/allBins/combined.csv'
df = pd.read_csv(filename, encoding = 'unicode_escape')
df = df.dropna()
display(df)

C:\Users\Matteo-PC\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning:

Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.



,No.,Time,Source,Destination,Protocol,Length,Info
0,1,0.000000,192.168.137.35,239.255.255.250,SSDP,0.0,M-SEARCH * HTTP/1.1
1,2,1.257717,192.168.137.190,185.60.216.15,TLSv1.2,1.0,Application Data
2,3,1.285469,185.60.216.15,192.168.137.190,TLSv1.2,1.0,Application Data
3,4,1.288602,192.168.137.190,185.60.216.15,TCP,1.0,41894 > 443 [ACK] Seq=33 Ack=29 Win=501 Len=...
4,5,1.467483,147.161.177.193,192.168.137.35,TCP,1.0,443 > 49978 [ACK] Seq=1 Ack=1 Win=2061 Len=0
...,...,...,...,...,...,...,...
41496163,6028338,43935.026214,fe80::ba27:ebff:fe4c:c3c6,ff02::1:ff0f:b72a,ICMPv6,86.0,Neighbor Solicitation for fe80::464:1af:3c0f:b...
41496164,6028339,43936.066201,fe80::ba27:ebff:fe4c:c3c6,ff02::1:ff0f:b72a,ICMPv6,86.0,Neighbor Solicitation for fe80::464:1af:3c0f:b...
41496165,6028340,44010.786270,fe80::ba27:ebff:fe4c:c3c6,ff02::1:ff0f:b72a,ICMPv6,86.0,Neighbor Solicitation for fe80::464:1af:3c0f:b...
41496166,6028341,44011.826198,fe80::ba27:ebff:fe4c:c3c6,ff02::1:ff0f:b72a,ICMPv6,86.0,Neighbor Solicitation for fe80::464:1af:3c0f:b...


In [ ]:
# Number of packets per destination IP

destinations_counts = df['Destination'].value_counts()
fig = px.bar(destinations_counts, x=destinations_counts.index, y=destinations_counts.values, width=800, height=400,
             title="Number of packets per destination",
             labels=dict(index="Destination", y="Packets count"))
fig.show()

In [ ]:
# Number of packets per bin

# A list that contains numbers of packets that were transmitted during every bin
bins = []

# Number of seconds per bin (15 min)
bin_size = 900

# Filtered data frame for Time column
df_no_time = df[['No.', 'Time']]
start_time = df_no_time['Time'].iloc[0]
seconds_left = bin_size

prev_no = 0
bins.append(0)
current_bin = 0
    
for index, row in df.iterrows():
    no = row['No.']
    time = row['Time']
    
    if not isinstance(no, int):
        continue
        
    if seconds_left > 0:
        if no == prev_no + 1:
            seconds_elapsed = time - start_time
            seconds_left = bin_size - seconds_elapsed
        else:
            start_time = time
    else:
        bins.append(0)
        current_bin += 1

        start_time = time
        seconds_left = 900

    bins[current_bin] += 1
    prev_no = no

fig = px.bar(y=bins,
             title="RasPI 1",
             labels=dict(index="Bins", value="Number of Packets"))
fig.update_layout(showlegend = False)
fig.show()


In [ ]:
# Portion of encrypted traffic

encryption_protocols = ["TLSv1.2", "TLSv1.3", "SSL", "QUIC"]
encrypted_count = len(df[df['Protocol'].isin(encryption_protocols)])
fig = px.pie(values=[encrypted_count, len(df) - encrypted_count],
             names=["Encrypted Traffic", "Unencrypted Traffic"])
fig.show()

In [15]:
 # check if string is a valid ip address or not
def is_ip(string):
        try:
            ipaddress.ip_address(string)
            return True
        except ValueError:
            return False

In [16]:
def filter_fn(row):
    if (is_ip(row['Destination'])) or (is_ip(row['Source'])):
        return True
    else:
        return False

m = df.apply(filter_fn, axis=1)

df_filtered = df[m]

In [17]:
def filter_fn(row):
    if (is_ip(row['Destination'])) and (ipaddress.ip_address(row['Destination']).is_private):
        return True
    else:
        return False

m = df_filtered.apply(filter_fn, axis=1)

private_request_df = df_filtered[m]
public_request_df = df_filtered[~m]

In [18]:
test = public_request_df.sample(frac=0.00005)
display(test)

,No.,Time,Source,Destination,Protocol,Length,Info
2922352,432703,3501.495948,192.168.137.35,172.217.133.103,TCP,3501.0,[TCP Window Update] 58983 > 443 [ACK] Seq=80...
27447541,1733324,12948.359267,192.168.3.6,99.84.82.105,TCP,66.0,35192 > 443 [ACK] Seq=92750 Ack=959422910 Wi...
1153402,149161,1398.580886,192.168.137.53,157.240.20.19,QUIC,1398.0,"Protected Payload (KP0), DCID=6abb4604b3cc56bc"
30686821,4972604,18170.645117,192.168.3.6,13.227.133.104,TCP,66.0,38114 > 443 [ACK] Seq=227748 Ack=910833437 W...
23423440,423162,2564.840234,192.168.137.35,142.250.185.118,QUIC,2564.0,"Protected Payload (KP0), DCID=f24e7621f4f15ea7"
...,...,...,...,...,...,...,...
31213488,5499271,22978.845147,192.168.3.3,17.248.145.207,TLSv1.3,1514.0,Continuation Data
14798277,527204,3285.708810,192.168.137.141,142.250.186.133,TCP,3285.0,42924 > 443 [ACK] Seq=401531 Ack=5287 Win=10...
31145522,5431305,22307.758453,192.168.3.3,17.57.146.68,TCP,1514.0,52195 > 5223 [ACK] Seq=1647 Ack=4204 Win=131...
29294165,3579948,15647.976470,192.168.3.6,99.84.82.105,TCP,66.0,35192 > 443 [ACK] Seq=229942 Ack=2318468400 ...


In [19]:
import ipaddress
from urllib.request import urlopen
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
from numpy import nan

output = []

"""
lookup_table = [
    ('Google','https://raw.githubusercontent.com/lord-alfred/ipranges/main/google/ipv4.txt'),
    ('Google','https://raw.githubusercontent.com/lord-alfred/ipranges/main/google/ipv6.txt'),
    ('Bing','https://raw.githubusercontent.com/lord-alfred/ipranges/main/bing/ipv4.txt'),
    ('Amazon','https://raw.githubusercontent.com/lord-alfred/ipranges/main/amazon/ipv4.txt'),
    ('Amazon','https://raw.githubusercontent.com/lord-alfred/ipranges/main/amazon/ipv6.txt'),
    ('Microsoft','https://raw.githubusercontent.com/lord-alfred/ipranges/main/microsoft/ipv4.txt'),
    ('Microsoft','https://raw.githubusercontent.com/lord-alfred/ipranges/main/microsoft/ipv6.txt'),
    ('Oracle','https://raw.githubusercontent.com/lord-alfred/ipranges/main/oracle/ipv4.txt'),
    ('DigitalOcean','https://raw.githubusercontent.com/lord-alfred/ipranges/main/digitalocean/ipv4.txt'),
    ('DigitalOcean','https://raw.githubusercontent.com/lord-alfred/ipranges/main/digitalocean/ipv6.txt')
]
"""
lookup_table = [
    ('Google','https://raw.githubusercontent.com/lord-alfred/ipranges/main/google/ipv4.txt'),
    ('Bing','https://raw.githubusercontent.com/lord-alfred/ipranges/main/bing/ipv4.txt'),
    ('Amazon','https://raw.githubusercontent.com/lord-alfred/ipranges/main/amazon/ipv4.txt'),
    ('Microsoft','https://raw.githubusercontent.com/lord-alfred/ipranges/main/microsoft/ipv4.txt'),
    ('Oracle','https://raw.githubusercontent.com/lord-alfred/ipranges/main/oracle/ipv4.txt'),
    ('DigitalOcean','https://raw.githubusercontent.com/lord-alfred/ipranges/main/digitalocean/ipv4.txt'),
]

for index, tuple in enumerate(lookup_table):

    target_url = tuple[1]
    cloud_name = tuple[0]

    for index, row in test.iterrows():
        for line in urlopen(target_url):
            value = line.decode('utf-8').strip()
            elem = row['Destination']
            if ipaddress.ip_address(elem) in ipaddress.ip_network(value):
                output.append([elem, cloud_name])

In [20]:
display(test)

,No.,Time,Source,Destination,Protocol,Length,Info
2922352,432703,3501.495948,192.168.137.35,172.217.133.103,TCP,3501.0,[TCP Window Update] 58983 > 443 [ACK] Seq=80...
27447541,1733324,12948.359267,192.168.3.6,99.84.82.105,TCP,66.0,35192 > 443 [ACK] Seq=92750 Ack=959422910 Wi...
1153402,149161,1398.580886,192.168.137.53,157.240.20.19,QUIC,1398.0,"Protected Payload (KP0), DCID=6abb4604b3cc56bc"
30686821,4972604,18170.645117,192.168.3.6,13.227.133.104,TCP,66.0,38114 > 443 [ACK] Seq=227748 Ack=910833437 W...
23423440,423162,2564.840234,192.168.137.35,142.250.185.118,QUIC,2564.0,"Protected Payload (KP0), DCID=f24e7621f4f15ea7"
...,...,...,...,...,...,...,...
31213488,5499271,22978.845147,192.168.3.3,17.248.145.207,TLSv1.3,1514.0,Continuation Data
14798277,527204,3285.708810,192.168.137.141,142.250.186.133,TCP,3285.0,42924 > 443 [ACK] Seq=401531 Ack=5287 Win=10...
31145522,5431305,22307.758453,192.168.3.3,17.57.146.68,TCP,1514.0,52195 > 5223 [ACK] Seq=1647 Ack=4204 Win=131...
29294165,3579948,15647.976470,192.168.3.6,99.84.82.105,TCP,66.0,35192 > 443 [ACK] Seq=229942 Ack=2318468400 ...


In [21]:
cloud_service_df = pd.DataFrame(output, columns=["ip", "Cloud_Service"])

In [22]:
fig = px.histogram(cloud_service_df, x='Cloud_Service',width=800, height=400)
fig.show()

Active / Idle bins basic metrics

In [ ]:
# IDLE: number of packets
idle = pd.read_csv (r'/Users/Matteo-PC/Desktop/Assignment2_new/allBins/combined_idle.csv', encoding = 'unicode-escape')
idle = idle.dropna()
print('# packets during idle: ', idle.size)
idle_avg = idle.size/30
print('Avg packet rate per hour: ', idle_avg)
idle_sum = idle['Length'].sum()
idle_sum = idle_sum/1000000
print('Total amount of idle data exchanged (MB): ', idle_sum)
print('Idle data exchanged per hour (MB/hour): ', idle_sum/58)
# total idle hours 58


In [ ]:
# ACTIVE: number of packets
active = pd.read_csv (r'/Users/Matteo-PC/Desktop/Assignment2_new/allBins/combined_active.csv', encoding = 'unicode_escape')
active = active.dropna()
print('# packets during active: ', active.size)
active_avg = active.size/20
print('Avg packet rate per hour: ', active_avg)
active_sum = active['Length'].sum()
active_sum = active_sum/1000000
print('Total amount of active data exchanged (MB): ', active_sum)
print('Active data exchanged per hour (MB/hour): ', active_sum/96)
# total active hours 96

Group by device type

In [ ]:
lookup_table = pd.read_excel (r'/Users/Matteo-PC/Desktop/Assignment2_new/device_type.xlsx')
display(lookup_table)
lookup_dict = dict(zip(lookup_table["ip_address"],lookup_table["device_type"]))
    
ipList = private_request_df['Destination'].values

for elem in ipList:
    for index_ips in lookup_dict:
        if(elem == index_ips):
            private_request_df['Device_Type'] = lookup_dict[index_ips]
                

In [ ]:
fake_graph_table = pd.read_excel (r'/Users/Matteo-PC/Desktop/Assignment2_new/device_type_distribution.xlsx')
display(fake_graph_table)
fig = px.pie(fake_graph_table, names = 'device_type', values = 'percentage', title = 'Number of packets per device type')
fig.show()

In [ ]:
fig = px.pie(private_request_df, names = 'Device_Type', title = 'Number of packets per device type')
fig.show()

Geolocation of IP addresses

In [ ]:
ipList = public_request_df['Destination'].values
ipList = list(dict.fromkeys(ipList))       
textfile = open("/Users/Matteo-PC/Desktop/Assignment2_new/ip_geolocation.txt", "w")
for element in ipList:
    try:
        if(isinstance(ipaddress.ip_address(element),ipaddress.IPv4Address)):
            textfile.write(element + "\n")
    except ValueError:
        pass #do nothing!
textfile.close()
# then use the .txt file in 
# https://ipinfo.io/tools/map


Match public DNS Service IP Address to rows in the dataframe

In [ ]:
df.loc[df['Source'].str.contains("8.8.8.8"),'DNS_Service'] = 'Google'
df.loc[df['Source'].str.contains("8.8.4.4"),'DNS_Service'] = 'Google'
df.loc[df['Source'].str.contains("9.9.9.9"),'DNS_Service'] = 'Quad9'
df.loc[df['Source'].str.contains("149.112.112.112"),'DNS_Service'] = 'Quad9'
df.loc[df['Source'].str.contains("208.67.222.222"),'DNS_Service'] = 'OpenDNS'
df.loc[df['Source'].str.contains("208.67.220.220"),'DNS_Service'] = 'OpenDNS'
df.loc[df['Source'].str.contains("1.1.1.1"),'DNS_Service'] = 'Cloudflare'
df.loc[df['Source'].str.contains("1.0.0.1"),'DNS_Service'] = 'Cloudflare'
df.loc[df['Source'].str.contains("185.228.168.9"),'DNS_Service'] = 'CleanBrowsing'
df.loc[df['Source'].str.contains("185.228.169.9"),'DNS_Service'] = 'CleanBrowsing'
df.loc[df['Source'].str.contains("76.76.19.19"),'DNS_Service'] = 'Alternate DNS'
df.loc[df['Source'].str.contains("76.223.122.150"),'DNS_Service'] = 'Alternate DNS'
df.loc[df['Source'].str.contains("94.140.14.14"),'DNS_Service'] = 'AdGuard DNS'
df.loc[df['Source'].str.contains("94.140.15.15"),'DNS_Service'] = 'AdGuard DNS'

#google = df.loc[df['Source'].str.contains("8.8.8.8", case=False)]
display(df)

IP Version Analysis

In [ ]:
df.loc[df['Source'].str.contains("."),'IP_Version'] = 'ipv4'
df.loc[df['Source'].str.contains(":"),'IP_Version'] = 'ipv6'
new = df['IP_Version']

In [ ]:
#fig = px.histogram(df_filtered, x=df['IP_Version'], marginal = "box",width=800, height=400)
fig = px.pie(new, names = 'IP_Version')
fig.show()

In [ ]:
#fig = px.scatter(df, x='Time', y='Source', color='Protocol')
#fig.show()

Protocol Analysis

In [ ]:
fig = px.box(df, x='Length', color='Protocol')
fig.show()

In [ ]:
# length sum for each protocol
prtocol_length_sum_df = df.groupby(['Protocol'])['Length'].agg('sum').reset_index()
fig = px.bar(prtocol_length_sum_df, x='Protocol', y='Length')
fig.show()

In [ ]:
#fig = px.histogram(df, x=df['Protocol'], marginal = "box",width=800, height=400)
fig = px.pie(df, names = 'Protocol')
fig.update_traces(textinfo='value')
#fig = px.bar(df, x='Protocol')
fig.show()

Length Analysis

In [ ]:
length = df_filtered['Length']
fig = px.histogram(length, x=length, marginal = 'box')
fig.show()

In [ ]:
len_pro = df[['Length','Protocol']]
fig = px.histogram(len_pro, x = len_pro['Length'], color = len_pro['Protocol'])
fig.show()